# 核心能力提升班商业智能方向 004期 Week 4

### Thinking 1： ALS都有哪些应用场景

ALS（Alternating Least Squares）交替最小二乘法，通常用于最优化矩阵分解问题，也可以应用在线性回归问题。

### Thinking 2： ALS进行矩阵分解的时候，为什么可以并行化处理

ALS在每次迭代时，固定用户因子矩阵或者是物品因子矩阵中的一个，然后用固定的这个矩阵以及评级数据来更新另一个矩阵。之后，被更新的矩阵被固定住，再更新另外一个矩阵。如此迭代，知道模型收敛（或者是迭代了预设好的次数）。因此在进行分解的时候可以分开计算。

### Thinking 3： 梯度下降法中的批量梯度下降（BGD），随机梯度下降（SGD），和小批量梯度下降有什么区别（MBGD）

批量梯度下降（BGD）在每次迭代时会使用所有样本进行梯度计算，这样的优点是可以计算全域梯度，使得参数更新的方向更靠近目标值，但缺点是计算量很大，训练过程太慢。  
随机梯度下降（SGD）在每次迭代时会使用一个样本进行梯度计算，这样的优点是计算速度很快，但缺点是损失震荡较大，并且可能无法收敛到全局最优。
小批量梯度下降（MBGD）是一个折中办法，每次迭代使用mini_batch_size个样本进行梯度计算，这样的优点是计算速度相较BGD快了很多，同时也能够使得收敛效果更接近BGD的效果，其缺点是需要人为设定mini_batch_size的大小，如果设置不当就无法发挥出MBGD的优势了。

### Thinking 4： 你阅读过和推荐系统/计算广告/预测相关的论文么？有哪些论文是你比较推荐的，可以分享到微信群中

我主要阅读的是基于深度学习的推荐系统相关论文，比较推荐的是Alibaba的几篇顶会论文，是非常有行业引领性的：   
1. Zhou, Guorui & Song, Chengru & Zhu, Xiaoqiang & Ma, Xiao & Yan, Yanghui & Dai, Xingya & Zhu, Han & Jin, Junqi & Li, Han & Gai, Kun. (2017). Deep Interest Network for Click-Through Rate Prediction. 
2. Zhou, Guorui & Mou, Na & Fan, Ying & Pi, Qi & Bian, Weijie & Zhou, Chang & Zhu, Xiaoqiang & Gai, Kun. (2018). Deep Interest Evolution Network for Click-Through Rate Prediction. 
3. Chen, Qiwei & Zhao, Huan & Li, Wei & Huang, Pipei & Ou, Wenwu. (2019). Behavior Sequence Transformer for E-commerce Recommendation in Alibaba. 
4. Sun, Fei & Liu, Jun & Wu, Jian & Pei, Changhua & Lin, Xiao & Ou, Wenwu & Jiang, Peng. (2019). BERT4Rec: Sequential Recommendation with Bidirectional Encoder Representations from Transformer. 

### Action 1： 对MovieLens数据集进行评分预测
工具：可以使用Surprise或者其他  
说明使用的模型，及简要原理  
数据集：MovieLens

In [1]:
from surprise import Dataset
from surprise import Reader
from surprise import BaselineOnly, SlopeOne
from surprise import accuracy
from surprise.model_selection import KFold

In [2]:
# 数据读取
reader = Reader(line_format='user item rating timestamp', sep=',', skip_lines=1)
data = Dataset.load_from_file('data/ratings.csv', reader=reader)
train_set = data.build_full_trainset()

#### BaselineOnly算法
分别应用了ALS优化方法和SGD优化方法  
$$ \hat{r_{ui}} = b_u + b_i + \mu  $$
Baseline算法是建立整体基线，然后根据user偏差和item偏差来得到评分。  
其中$\mu$可以通过统计得到，而$b_u$和$b_i$可以通过ALS或SGD得到。  
<b>ALS</b>中文名为交替最小二乘法，是求解矩阵分解问题的一种最优化方法，其原理是迭代式求解一系列最小二乘回归问题。在每次迭代时，固定用户因子矩阵或者是物品因子矩阵中的一个，然后用固定的这个矩阵以及评级数据来更新另一个矩阵。之后，被更新的矩阵被固定住，再更新另外一个矩阵。如此迭代，知道模型收敛（或者是迭代了预设好的次数）。  
<b>SGD</b>中文名为随机梯度下降，是一种常用的最优化方法，其原理是迭代的进行参数更新，每次迭代只用一个训练数据进行梯度计算，这样带来的优势是计算速度快，但缺点是在训练过程中会造成损失震荡幅度较大。

In [3]:
# ALS优化
bsl_options = {'method': 'als','n_epochs': 5,'reg_u': 12,'reg_i': 5}
algo = BaselineOnly(bsl_options=bsl_options)

In [4]:
# 定义K折交叉验证迭代器，K=3
kf = KFold(n_splits=3)
for trainset, testset in kf.split(data):
    # 训练并预测
    algo.fit(trainset)
    predictions = algo.test(testset)
    # 计算RMSE
    accuracy.rmse(predictions, verbose=True)

Estimating biases using als...
RMSE: 0.8651
Estimating biases using als...
RMSE: 0.8643
Estimating biases using als...
RMSE: 0.8618


In [5]:
uid = str(196)
iid = str(302)
# 输出uid对iid的预测结果
pred = algo.predict(uid, iid, r_ui=4, verbose=True)

user: 196        item: 302        r_ui = 4.00   est = 4.14   {'was_impossible': False}


In [6]:
# SGD优化
bsl_options = {'method': 'sgd','n_epochs': 5}
algo = BaselineOnly(bsl_options=bsl_options)
# 定义K折交叉验证迭代器，K=3
kf = KFold(n_splits=3)
for trainset, testset in kf.split(data):
    # 训练并预测
    algo.fit(trainset)
    predictions = algo.test(testset)
    # 计算RMSE
    accuracy.rmse(predictions, verbose=True)
uid = str(196)
iid = str(302)
# 输出uid对iid的预测结果
pred = algo.predict(uid, iid, r_ui=4, verbose=True)

Estimating biases using sgd...
RMSE: 0.8754
Estimating biases using sgd...
RMSE: 0.8732
Estimating biases using sgd...
RMSE: 0.8741
user: 196        item: 302        r_ui = 4.00   est = 4.00   {'was_impossible': False}


#### SlopeOne算法
SlopeOne算法基于user之间和item之间的评分差异来进行评分预测的。其大致分为三步：
1. 计算物品之间的评分差的均值，记为物品间的评分偏差(两物品同时被评分)
2. 根据物品间的评分偏差和用户的历史评分，预测用户对未评分的物品的评分
3. 将预测评分排序，取topN对应的物品推荐给用户

In [7]:
# 使用SlopeOne算法
algo2 = SlopeOne()
# 定义K折交叉验证迭代器，K=3
kf = KFold(n_splits=3)
for trainset, testset in kf.split(data):
    # 训练并预测
    algo2.fit(trainset)
    predictions = algo2.test(testset)
    # 计算RMSE
    accuracy.rmse(predictions, verbose=True)

RMSE: 0.8671
RMSE: 0.8677
RMSE: 0.8679


In [8]:
# 输出uid对iid的预测结果
pred = algo2.predict(uid, iid, r_ui=4, verbose=True)

user: 196        item: 302        r_ui = 4.00   est = 4.31   {'was_impossible': False}


### Action 2： Paper Reading：Slope one predictors for online rating-based collaborative filtering. Daniel Lemire and Anna Maclachlan, 2007. http://arxiv.org/abs/cs/0702144.
积累，总结笔记，自己的思考及idea


#### 1 Introduction
论文作者提出鲁棒性的CF应该符合以下几点：
1. 算法容易实现和维护
2. 对新的评分应该立即给予响应
3. 查询速度要快
4. 对新的用户也要能给出有效的推荐
5. 在不做出重大牺牲的前提下精度上要有竞争力

Slope One的核心思想是利用用户之间的评分差异和商品之间的人气差异（popularity differential）  
<img src="Figure 1.png">  
此文章的主要贡献是提出了Slope One协同过滤方法，并且展示了其相比memory-based的方法在CF任务上具有几乎相同的精度。

#### 2 Related Work
主要介绍基于内存的和基于模型的几种方法。

#### 3 CF Algorithms
首先引出了作为对比的四个其他方案： <b>PER USER AVERAGE, BIAS FROM MEAN, ADJUSTED COSINE ITEM-BASED and The PEARSON Reference Scheme</b> 
1. PER USER AVERAGE: 一个简单粗暴的方案，直接将user对所有item的评分的平均值作为user对于新item的评分。
$$P(u) = \bar u $$
2. BIAS FROM MEAN：在user评分均值的基础上加入所有user对item的评分偏差。  
$$ P(u)_{i} = \bar{u} + \frac{1}{card(S_{i}(\chi ))}\sum_{\nu\in S_{i}(\chi )}\nu_{i}-\bar{\nu} $$   
其中，$P(u)_{i}$是用户u对商品i的评分， $ S_{i}(\chi )$是有对$i$作过评分的用户的集合，$\nu_{i}$是用户$v$对商品$i$的评分，$card$是其内集合的元素项的个数。
3. ADJUSTED COSINE ITEM-BASED：通过user对item的评分来计算item间的相似度，从而推荐相似的商品给用户。
$$sim_{i,j} = \frac{\sum_{u\in S_{i,j}(\chi )}(u_{i}-\bar{u})(u_{j}-\bar{u})}{\sqrt{\sum_{u\in S_{i,j}(\chi )}(u_{i}-\bar{u})^{2}\sum_{u\in S_{i,j}(\chi )}(u_{j}-\bar{u})^{2}}}$$
$$P(u)_{i} = \frac{\sum _{j\in S(u)}\left | sim_{i,j} \right | (\alpha _{i,j}u_{j}+\beta _{i,j})}{\sum _{j\in S(u)}\left | sim_{i,j} \right |}$$
其中，$S_{i,j}(\chi )$是有对$i$,$j$都作过评分的用户的集合，$sim_{i,j}$计算商品$i$,$j$间的相似度，$P(u)_{i}$即用户u对商品i的评分，$\alpha$和$\beta$是校正因子。
4. The PEARSON Reference Scheme：是一个基于内存的算法，在BIAS FROM MEAN算法的基础上引入了user间的相似度作为权重。
$$Corr(u,w)=\frac{< u-\bar u,w-\bar w > }{\sqrt{\sum_{i\in S(u)\bigcap S(w)}(u_{i}-\bar{u})^{2}\sum_{i\in S(u)\bigcap S(w)}(w_{i}-\bar{w})^{2}}}$$
$$\gamma (u,w) = Corr(u,w)\left | Corr(u,w) \right |^{\rho -1}$$
$$P(u)_{i} = \bar{u} + \frac{\sum _{\nu \in S_i (\chi )}\gamma (u,\nu )(\nu _i - \bar \nu)}{\sum _{\nu \in S_i (\chi )}\left | \gamma (u,\nu ) \right |}$$
其中，$\gamma(u,\nu)$ 是用户$u$,$\nu$之间的相似度。

本文提出了三个方案：<b>The SLOPE ONE Scheme, The WEIGHTED SLOPE ONE Scheme, The BI-POLAR SLOPE ONE Scheme</b>
1. The SLOPE ONE Scheme：用item之间的评分差异的均值来代表item之间的差别，用user之间的评分差异的均值来代表user之间的差别，根据user之间的差异和item之间的差异来计算待求评分。
$$dev_{j,i} = \sum_{u\in S_{j,i}(\chi )}\frac{u_j - u_i}{card(S_{j,i}(\chi ))}$$
$$P(u)_j = \frac{1}{card(R_j)}\sum_{i\in R_j}(dev_{j,i}+u_i)$$
$$P^{S1}(u)_j = \bar u + \frac{1}{card(R_j)}dev_{j,i}$$
其中，$dev_{j,i}$表示item i,j之间的评分差异，$P(u)_j$通过平均不同商品的评分差距可以得到user i对item j的评分。
2. The WEIGHTED SLOPE ONE Scheme：Slope One中对于不同的评分是一视同仁的，但事实上不同的评分的可信度不同，通过引入权重提高可信的评分的影响（被评分较多的item的评分可信度较高）。
$$P^{wS1}(u)_j = \frac{\sum_{i \in S(u)-\left \{ j \right \}}(dev_{j,i}+u_i)c_{j,i}}{\sum_{i \in S(u)-\left \{ j \right \}}c_{j,i}}$$
其中，$c_{j,i} = card(S_{j,i}(\chi ))$
3. The BI-POLAR SLOPE ONE Scheme：从user对item i,j的评分中筛选出喜欢item i,j的评分和不喜欢item i,j的评分（其中喜欢和不喜欢的定义是评分高于或低于user的平均评分），只用这些评分来计算item i,j之间的评分差异，这样减少了个人喜好对于评分的影响。
$$P^{bpS1}(u)_{j} = \frac{\sum_{i \in S^{like}(u)-\left \{ j \right \}}p_{j,i}^{like}c_{j,i}^{like}+\sum_{i \in S^{diskile}(u)-\left \{ j \right \}}p_{j,i}^{dislike}c_{j,i}^{dislike}}{\sum_{i \in S^{like}(u)-\left \{ j \right \}}c_{j,i}^{like}+\sum_{i \in S^{dislike}(u)-\left \{ j \right \}}c_{j,i}^{dislike}}$$
其中，$c_{j,i}^{like} = card(S_{j,i}^{like})$, $c_{j,i}^{dislike} = card(S_{j,i}^{dislike})$, $S^{like}(u) = \left \{ i\in S(u)|u_{i}>\bar{u} \right \}$, $S^{dislike}(u) = \left \{ i\in S(u)|u_{i}<\bar{u} \right \}$

#### 4 Experimental Results
<img src="Table 1.png">  

#### <b>自己的思考</b>
Slope One算法的核心思想是计算user之间和item之间的区别来“类比”的推演出待评分的商品的评分，其中BI-POLAR SLOPE ONE算法通过筛选方式减少个人喜好对于评分的影响，所以在此基础上也应该可以应用K-Means算法先将user和item进行聚类，然后分别计算同类之间的评分差距（就像BI-POLAR SLOPE ONE中分别计算like和dislike那样），这样可以可以进一步减少user或item类别差异对评分差距造成的偏差。

### Action 3： 设计你自己的句子生成器
grammar = '''  
战斗 => 施法  ， 结果 。  
施法 => 主语 动作 技能   
结果 => 主语 获得 效果  
主语 => 张飞 | 关羽 | 赵云 | 典韦 | 许褚 | 刘备 | 黄忠 | 曹操 | 鲁班七号 | 貂蝉  
动作 => 施放 | 使用 | 召唤   
技能 => 一骑当千 | 单刀赴会 | 青龙偃月 | 刀锋铁骑 | 黑暗潜能 | 画地为牢 | 守护机关 | 狂兽血性 | 龙鸣 | 惊雷之龙 | 破云之龙 | 天翔之龙
获得 => 损失 | 获得   
效果 => 数值 状态  
数值 => 1 | 1000 |5000 | 100   
状态 => 法力 | 生命  
'''  

In [9]:
import random

# 定语从句语法
grammar = '''
战斗 => 施法  ， 结果 。
施法 => 主语 动作 技能 
结果 => 主语 获得 效果
主语 => 张飞 | 关羽 | 赵云 | 典韦 | 许褚 | 刘备 | 黄忠 | 曹操 | 鲁班七号 | 貂蝉
动作 => 施放 | 使用 | 召唤 
技能 => 一骑当千 | 单刀赴会 | 青龙偃月 | 刀锋铁骑 | 黑暗潜能 | 画地为牢 | 守护机关 | 狂兽血性 | 龙鸣 | 惊雷之龙 | 破云之龙 | 天翔之龙
获得 => 损失 | 获得 
效果 => 数值 状态
数值 => 1 | 1000 |5000 | 100 
状态 => 法力 | 生命
'''


In [10]:
# 得到语法字典
def getGrammarDict(gram, linesplit = "\n", gramsplit = "=>"):
    #定义字典
    result = {}

    for line in gram.split(linesplit):
        # 去掉首尾空格后，如果为空则退出
        if not line.strip(): 
            continue
        expr, statement = line.split(gramsplit)
        result[expr.strip()] = [i.split() for i in statement.split("|")]
    #print(result)
    return result

In [11]:
gramdict = getGrammarDict(grammar)
gramdict

{'战斗': [['施法', '，', '结果', '。']],
 '施法': [['主语', '动作', '技能']],
 '结果': [['主语', '获得', '效果']],
 '主语': [['张飞'],
  ['关羽'],
  ['赵云'],
  ['典韦'],
  ['许褚'],
  ['刘备'],
  ['黄忠'],
  ['曹操'],
  ['鲁班七号'],
  ['貂蝉']],
 '动作': [['施放'], ['使用'], ['召唤']],
 '技能': [['一骑当千'],
  ['单刀赴会'],
  ['青龙偃月'],
  ['刀锋铁骑'],
  ['黑暗潜能'],
  ['画地为牢'],
  ['守护机关'],
  ['狂兽血性'],
  ['龙鸣'],
  ['惊雷之龙'],
  ['破云之龙'],
  ['天翔之龙']],
 '获得': [['损失'], ['获得']],
 '效果': [['数值', '状态']],
 '数值': [['1'], ['1000'], ['5000'], ['100']],
 '状态': [['法力'], ['生命']]}

In [12]:
# 生成句子
def generate(gramdict, target, isEng = False):
    if target not in gramdict: 
        return target
    find = random.choice(gramdict[target])
    #print(find)
    blank = ''
    # 如果是英文中间间隔为空格
    if isEng: 
        blank = ' '
    return blank.join(generate(gramdict, t, isEng) for t in find)

In [13]:
print(generate(gramdict,"战斗"))
print(generate(gramdict,"战斗", True))

曹操召唤天翔之龙，貂蝉损失1法力。
许褚 施放 破云之龙 ， 张飞 损失 1000 生命 。


In [14]:
host = """
host = 寒暄 报数 询问 具体业务 结尾 
报数 = 我是工号 数字 号 ,
数字 = 单个数字 | 数字 单个数字 
单个数字 = 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 
寒暄 = 称谓 打招呼 | 打招呼
称谓 = 人称 ,
人称 = 先生 | 女士 | 小朋友
打招呼 = 你好 | 您好 
询问 = 请问你要 | 您需要
具体业务 = 喝酒 | 打牌 | 打猎 | 赌博
结尾 = 吗？
"""

In [15]:
hostdict = getGrammarDict(host, linesplit = "\n", gramsplit = "=")
hostdict

{'host': [['寒暄', '报数', '询问', '具体业务', '结尾']],
 '报数': [['我是工号', '数字', '号', ',']],
 '数字': [['单个数字'], ['数字', '单个数字']],
 '单个数字': [['1'], ['2'], ['3'], ['4'], ['5'], ['6'], ['7'], ['8'], ['9']],
 '寒暄': [['称谓', '打招呼'], ['打招呼']],
 '称谓': [['人称', ',']],
 '人称': [['先生'], ['女士'], ['小朋友']],
 '打招呼': [['你好'], ['您好']],
 '询问': [['请问你要'], ['您需要']],
 '具体业务': [['喝酒'], ['打牌'], ['打猎'], ['赌博']],
 '结尾': [['吗？']]}

In [17]:
print(generate(hostdict,"host"))
print(generate(hostdict,"host"))

您好我是工号25号,您需要喝酒吗？
小朋友,您好我是工号7号,请问你要打猎吗？
